# skyscraper

In [193]:
# Implement an algorithm to find the first element greater than a given amount
# design a query algorithm that will find the first element greater than a given amount
# max segment tree
import math
class SegmentTree:
    def __init__(self,arr):
        self.arr = arr
        n = len(arr)
        self.neutral = math.inf
        self.size = 1
        while self.size<n:
            self.size*=2
        self.tree = [0 for _ in range(self.size*2)]
    def update_tree(self, idx, val):
        self.update(idx,val,0,0,self.size)
    def build_tree(self):
        for i, val in enumerate(self.arr):
            self.update_tree(i,val)
    def update(self,idx,val,x,lx,rx):
        if rx-lx==1:
            self.tree[x] = val
            return
        mid = rx+lx>>1
        if idx<mid:
            self.update(idx,val,2*x+1,lx,mid)
        else:
            self.update(idx,val,2*x+2,mid,rx)
        self.tree[x] = max(self.tree[2*x+1],self.tree[2*x+2])
    def get_first_tree(self,l, r,val):
        return self.get_first(l,r,0,0,self.size,val)
    def get_first(self,l,r,x,lx,rx,val):
        if lx>=r or rx<=l: return -1
        if l<=lx and rx<=r:
            if self.tree[x] <= val: return -1
            while rx != lx+1:
                mid = lx + rx >> 1
                if self.tree[2*x+1]>val:
                    x = 2*x+1
                    rx = mid
                else:
                    x = 2*x+2
                    lx = mid

            return lx
        mid = lx+rx>>1
        left_segment = self.get_first(l,r,2*x+1,lx,mid,val)
        if left_segment != -1: return left_segment
        return self.get_first(l,r,2*x+2,mid,rx,val)
    def get_last_tree(self,l,r,val):
        return self.get_last(l,r,0,0,self.size,val)
    def get_last(self,l,r,x,lx,rx,val):
        if lx>=r or rx<=l: return -1
        if l<=lx and rx<=r:
            if self.tree[x] <= val: return -1
            while rx != lx+1:
                mid = lx+rx>>1
                if self.tree[2*x+2]>val:
                    x=2*x+2
                    lx=mid
                else:
                    x=2*x+1
                    rx=mid
            return lx
        mid = lx+rx>>1
        right_segment = self.get_last(l,r,2*x+2,mid,rx,val)
        if right_segment != -1: return right_segment
        return self.get_last(l,r,2*x+1,lx,mid,val)
    def get_count(self,i):
        left_index = self.get_last_tree(0,i+1,self.arr[i])
        right_index = self.get_first_tree(i+1,n,arr[i])
        right_index = right_index if right_index!=-1 else len(self.arr)
        return right_index-left_index-1

In [194]:
arr = [4,2,3,2,4,7,6,5]
n = len(arr)
segment_tree = SegmentTree(arr)
segment_tree.build_tree()

In [195]:
segment_tree.tree

[7, 4, 7, 4, 3, 7, 6, 4, 2, 3, 2, 4, 7, 6, 5, 0]

In [196]:
def get_size(i):
    left_index = segment_tree.get_last_tree(0,i+1,arr[i])
    right_index = segment_tree.get_first_tree(i+1,n,arr[i]) 
    right_index = right_index if right_index!=-1 else n
    return right_index-left_index-1

In [205]:
import unittest
class testQuery(unittest.TestCase):
    def test(self):
        arr = [4,2,3,2,4,7,6,5]
        segment_tree = SegmentTree(arr)
        segment_tree.build_tree()
        self.assertEqual(segment_tree.get_count(0),5)
        self.assertEqual(segment_tree.get_count(1),1)
        self.assertEqual(segment_tree.get_count(2),3)
        self.assertEqual(segment_tree.get_count(4),5)
        self.assertEqual(segment_tree.get_count(5),8)
        self.assertEqual(segment_tree.get_count(6),2)
        segment_tree.update_tree(2,8)
        self.assertEqual(segment_tree.get_count(4),2)
        for i in range(4,7):
            segment_tree.update_tree(i,1)
        self.assertEqual(segment_tree.get_count(7),5)

In [206]:
unittest.main(argv=[''], verbosity=3, exit=False)

test (__main__.testQuery) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


# TODO: Add lazy propagation to improve time complexity

Right now let's look at time complexity
so num_visible_skyscraper(i) is O(logn), because it just performs a search for th left and right index in segment tree.
set_height(i,x) is O(logn) because it just updates a single element
set_heigth_range(i,j,x) is O(nlogn) because it could iterate through n elements to update all of them. 

With lazy propagation we can change all the update queries, even the range one to O(logn), 
However the query of num_visible_skyscraper(i) could still be nlogn if it needs to update all the values. 